In [1]:
%load_ext jupyter_black

In [6]:
import json
from glob import glob
from pathlib import Path
from typing import Iterator, NewType, Union
from datetime import datetime
from warnings import warn
import pandas as pd
import numpy as np
from geopandas import GeoDataFrame

from sppp.schema import FeatureCollection

idx: slice = pd.IndexSlice

In [9]:
def iterpaths() -> Iterator[Path]:
    yield from (Path(file) for file in glob("/workspaces/sppp/data/new/*.json"))


def to_midf() -> pd.DataFrame:
    def generate():
        for path in iterpaths():
            with path.open("rb") as f:
                try:
                    fc: FeatureCollection = json.load(f)
                except json.JSONDecodeError:
                    warn(f"json error decoding {path}" )
                    continue
                df: GeoDataFrame = GeoDataFrame.from_features(fc["features"])
                df["validTime"] = datetime.strptime(fc["validTime"], "%Y%m%d_%H%M%S %Z")
                yield df

    return pd.concat(generate()).set_index(["validTime", "ID"])


df = to_midf()
df

geometry  \
validTime           ID                                                         
2021-10-21 21:12:57 5048   POLYGON ((-78.78000 41.35000, -78.71000 41.350...   
                    5134   POLYGON ((-86.36000 35.20000, -86.34000 35.200...   
                    5233   POLYGON ((-84.94000 36.45000, -84.87000 36.450...   
                    5264   POLYGON ((-103.98000 30.45000, -103.97000 30.4...   
                    5342   POLYGON ((-87.39000 34.05000, -87.38000 34.040...   
...                                                                      ...   
2021-10-12 05:24:55 99608  POLYGON ((-85.81000 42.59000, -85.77000 42.590...   
                    99609  POLYGON ((-85.79000 42.45000, -85.78000 42.450...   
                    99612  POLYGON ((-85.53000 46.71000, -85.49000 46.710...   
                    99616  POLYGON ((-87.77000 34.90000, -87.75000 34.860...   
                    99617  POLYGON ((-85.07000 42.00000, -85.03000 42.000...   

                          MUCAPE MLCAPE MLCIN EBSHEAR SRH01KM  \
validTime           ID                                          
2021-10-21 21:12:57 5048     524    381    -5    33.2     144   
                    5134    1707   1384     0    31.7      75   
                    5233    1666   1136  -999    32.7     112   
                    5264    1009    782   -25    20.5      17   
                    5342    1946   1606    -1    26.5      39   
...                          ...    ...   ...     ...     ...   
2021-10-12 05:24:55 99608    215    163    -9    39.2     152   
                    99609    229    188    -3    40.0     178   
                    99612    855    297   -21    34.0      77   
                    99616    263    131  -264    30.2     162   
                    99617    295    186   -83    32.3     356   

                          MEANWIND_1-3kmAGL  MESH VIL_DENSITY FLASH_RATE  ...  \
validTime           ID                                                    ...   
2021-10-21 21:12:57 5048               38.0  0.05        1.09          0  ...   
                    5134               27.4  0.16        1.16          2  ...   
                    5233               38.1  0.11        1.13          0  ...   
                    5264                6.9  0.44        0.84          2  ...   
                    5342               24.8  0.03        1.66          0  ...   
...                                     ...   ...         ...        ...  ...   
2021-10-12 05:24:55 99608              37.5  0.00        0.43          0  ...   
                    99609              38.2  0.00        1.49          0  ...   
                    99612              32.6  0.00        0.38          0  ...   
                    99616              29.0  0.00        0.35          0  ...   
                    99617              46.1  0.00        0.36          0  ...   

                                         MAXRC_ICECF WETBULB_0C_HGT PWAT  \
validTime           ID                                                     
2021-10-21 21:12:57 5048                         N/A            6.5  1.1   
                    5134                         N/A            9.4  1.5   
                    5233     1901Z 0.12/min (strong)            8.7  1.3   
                    5264                         N/A            7.7  0.8   
                    5342   1911Z 0.04/min (moderate)            9.7  1.4   
...                                              ...            ...  ...   
2021-10-12 05:24:55 99608                        N/A           10.2  1.7   
                    99609                        N/A           10.5  1.7   
                    99612                        N/A           10.3  1.7   
                    99616                        N/A           11.7  1.4   
                    99617                        N/A           10.5  1.5   

                          CAPE_M10M30  LJA SIZE         AVG_BEAM_HGT  \
validTime           ID                                            

In [13]:
#
df.loc[df["PS"].astype(float) > 90, "PS"]
# df.loc[idx[:, "89519"], :]
# import pyarrow

validTime            ID    
2021-10-13 21:10:41  110381     92
2021-10-15 19:02:44  118692     96
2021-10-12 23:58:40  101533     99
                     101613     99
                     101941     99
                              ... 
2021-10-13 05:12:36  103365     94
                     103476     95
2021-10-13 00:36:40  101533     98
                     101613     97
                     101941    100
Name: PS, Length: 1007, dtype: object